In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
from collections import Counter

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# import pre-cleaned data

base_df = pd.read_csv('data/rtr_data.csv')
base_df.drop(columns='Unnamed: 0', inplace=True) # damn this mistake
base_df.dropna(inplace=True) # for the review-summary data

In [4]:
f'unique items: {len(base_df["item_id"].unique())}, unique users: {len(base_df["user_id"].unique())}'

'unique items: 5734, unique users: 77237'

In [5]:
# user_id, item_id, rating

base_rating_df = base_df[['user_id', 'item_id', 'rating']]

In [6]:
base_rating_df

,user_id,item_id,rating
0,420272,2260466,10.0
1,273551,153475,10.0
2,909926,126335,8.0
3,151944,616682,10.0
4,734848,364092,8.0
...,...,...,...
146132,66386,2252812,10.0
146133,118398,682043,10.0
146134,47002,683251,6.0
146135,961120,126335,10.0


In [ ]:
base_rating_utility = base_rating_df.pivot_table(values='rating', index='user_id', columns='item_id')
base_rating_utility.fillna(base_rating_utility.mean(), inplace=True)

In [ ]:
base_rating_utility

In [ ]:
itemid = 2953681

temp = cosine_similarity(base_rating_utility[itemid].values.reshape(1,-1), base_rating_utility.T)
sim_item_id = np.argsort(temp)[0][::-1][:10]
sim_item_id

In [ ]:
base_df.iloc[4326]['category']

In [ ]:
cat_lst = []

for idx in sim_item_id:
  cat_lst.append(base_df.iloc[idx]['category'])

cat_lst

In [ ]:
# some sort of random choice; 60% of the time it's okay to recommend dress,gown,sheath, 40% recommend the other options
# since it's expected to rent a number of garments at a time, top 'n' picks but only let 50% be dress,gown,sheath
# refer to Capstone II to use review-summary keywords, category, rented_for, body_type for item-item recommender

In [ ]:
# get_dummies function

def get_dummies(df, lst):
  result = []
  for col in lst:
    temp = pd.get_dummies(df[col])
    result.append(temp)
  return pd.concat(result, axis=1)

In [ ]:
get_dummies_list = ['category', 'rented_for', 'body_type']
basedf_dummies = get_dummies(base_df, get_dummies_list)

In [ ]:
basedf_dummies

In [ ]:
# one hot encoder?

item_dummy = pd.concat([base_df['item_id'], basedf_dummies], axis=1)

In [ ]:
item_dummy.replace(0,2, inplace=True)

In [ ]:
item_bools = item_dummy.groupby('item_id').min()

In [ ]:
item_bools.replace(2,0, inplace=True)
item_bools

In [ ]:
# cosine_similarity(item_bools)

In [ ]:
item_cosine = pd.DataFrame(cosine_similarity(item_bools), index = item_bools.index, columns = item_bools.index)
item_cosine

In [ ]:
test3 = list(item_cosine[item_cosine[2953965] < 0.99][2953965].sort_values(ascending=False).head(10).index)
test3

In [ ]:
base_df[base_df['item_id'] == 2134815].iloc[0]